## Lookuptable experiment

In [1]:
%load_ext autoreload
%autoreload 2
%env CUDA_VISIBLE_DEVICES=3
!echo $CUDA_VISIBLE_DEVICES

env: CUDA_VISIBLE_DEVICES=3
3


In [2]:
## discovery cluster setup
# CACHE_DIR = "/scratch/lucchetti.f/models/"
# os.environ['TRANSFORMERS_CACHE'] = CACHE_DIR

import os
import sys
sys.path.append('..')

import random
import gc
import torch
import pandas as pd
from model_utils import *
import tqdm as notebook_tqdm
from trace_model import ModelLoader
from pandas import DataFrame
torch.set_grad_enabled(False)
from tqdm import tqdm 
from model_utils import layername
import numpy as np
from extract_data import *
import nltk
nltk.download('words')
from random import sample
from patch import *
import json 

import re
check_devs()
# clear_devs()
torch.cuda.is_available()

MODEL_NAME = "/home/arjun/models/starcoderbase-1b"

/home/franlucc/mechinterp/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


0 / 84978434048 used for device 0, reserved 0


[nltk_data] Downloading package words to /home/franlucc/nltk_data...
[nltk_data]   Package words is already up-to-date!


In [3]:
lm = ModelLoader(MODEL_NAME)
check_devs()

4615938048 / 84978434048 used for device 0, reserved 4617928704


In [12]:
lm.model.config

GPTBigCodeConfig {
  "_name_or_path": "/home/arjun/models/starcoderbase-1b",
  "activation_function": "gelu_pytorch_tanh",
  "architectures": [
    "GPTBigCodeForCausalLM"
  ],
  "attention_softmax_in_fp32": true,
  "attn_pdrop": 0.1,
  "bos_token_id": 0,
  "embd_pdrop": 0.1,
  "eos_token_id": 0,
  "inference_runner": 0,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "max_batch_size": null,
  "max_sequence_length": null,
  "model_type": "gpt_bigcode",
  "multi_query": true,
  "n_embd": 2048,
  "n_head": 16,
  "n_inner": 8192,
  "n_layer": 24,
  "n_positions": 8192,
  "output_attentions": true,
  "pad_key_length": true,
  "pre_allocate_kv_cache": false,
  "resid_pdrop": 0.1,
  "scale_attention_softmax_in_fp32": true,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "torch_dtype": "float32",
  "transformers_version": "4.28.0",
  "use_cac

## synthetic look-up table

In [10]:

prompt = """def func(i : int) -> str:
    if i == 0:
        return "cat"
    elif i == 1:
        return "speed"
    elif i == 3:
        return "city"
    else:
        return "number"
"""

def make_assert(n):
    return f"assert func({n}) == \""

assertions = [
    "assert func(0) == \"",
    "assert func(1) == \"",
    "assert func(2) == \"",
    "assert func(3) == \"",
    "assert func(999) == \"",
    
]

def layernum(lname):
    return int(lname.split(".")[-1])

### Step 1: prune transformer, get earliest layer that successfully completes task

problem: sampling is fast, greedy is slow

In [ ]:
txt, ret_dict = lm.trace_generate(prompt + "\n" + make_assert(99),
                                  max_new_toks=1,
                                  request_logits=[layername(lm.model, l) for l in range(10,30)],
                                  sample_from_topk_logits=None)
out = txt.replace(prompt + "\n" + make_assert(99), "")


In [ ]:
print(out)
ret_dict["logits"]

In [4]:
def trace_lookuptable(max_n, prompt, sample_from=None):
    res = []
    exec(prompt)
    for n in range(0, max_n):
        # for k in range(0, 20):
            txt, ret_dict = lm.trace_generate(prompt + "\n" + make_assert(n),
                                            max_new_toks=1,
                                            request_logits=[layername(lm.model, l) for l in range(10,30)],
                                            sample_from_topk_logits=sample_from)
            out = txt.replace(prompt + "\n" + make_assert(n), "")
            for lname, logits in ret_dict["logits"].items():
                if out in [str(x) for (x,_) in logits]:
                    gold = eval(f"func({n})")
                    res.append(((out == gold), out, gold, n, layernum(lname)))
                    break
    return res

## Try to break lookup table task

make harder by increasing lenght. At what point is transformer unsure what task this is?

In [5]:
def save_to_csv(res, name, header=['prompt_seed','is_correct','completion', 'gold', 'input n', 'layernum']):
    ds = DataFrame.from_dict(res)
    ds.to_csv(f"../results/{name}.csv", header=header)

In [6]:
def make_lookup_table(n, seed=0):
    assert n < 25, "n must be less than 25"
    random.seed(seed)
    nums = sample(list(range(n)), n)
    words = sample(list(range(n+1)), n+1)
    s = f"def func(i : int) -> str:\n\tif i == {nums[0]}:\n\t\treturn \"{chr(97+words[0])}\"\n"
    for i,k in enumerate(nums[1:]):
        s += f"\telif i == {k}:\n\t\treturn \"{chr(97+words[i+1])}\"\n"
    s += f"\telse:\n\t\treturn \"{chr(97+words[-1])}\""
    return s.replace("\t","    ")

In [13]:
def trace_false_task(max_n, prompt, seed, len_table=12):
    res = []
    exec(prompt)
    
    for n in range(0, max_n):
        # for k in range(0, 20):
            txt, ret_dict = lm.trace_generate(prompt + "\n" + make_assert(n),
                                            max_new_toks=1,
                                            request_logits=[layername(lm.model, l) for l in range(10,20)],
                                            do_greedy_decoding=True)
            out = txt.replace(prompt + "\n" + make_assert(n), "")
            for lname, logits in ret_dict["logits"].items():
                if out in [str(x) for (x,_) in logits]:
                    gold = eval(f"func({n})")
                    def first_or_last_match(out, i):
                        exec(prompt)
                        first_digit = eval(f"func({str(i)[0]})")
                        last_digit = eval(f"func({str(i)[-1]})")
                        if first_digit == out:
                            return "Match_First_Digit_False_Task"
                        elif last_digit == out:
                            return "Match_Last_Digit_False_Task"
                        else:
                            return "Neither_False_Task"
                    is_correct = (out == gold)
                    first_or_last_match_str = first_or_last_match(out, n) if not is_correct else "NA"
                    res.append((len_table, seed, is_correct, first_or_last_match_str, out, gold, n, layernum(lname)))
                    break
    return res

In [8]:
import json
def exp_false_task(seed_range, lengths_table=[3,6,9,12], seed_start=0):
    fp = open(f"../results/log.json", "w")
    tot_res = []
    for len_table in tqdm(lengths_table):
        for seed in range(seed_start, seed_range):
            prompt = make_lookup_table(len_table, seed=seed)
            res = trace_false_task(50, prompt, seed, len_table=len_table)
            json.dump(res, fp)
            tot_res += res
    fp.close()
    return tot_res


In [14]:
tot_res = exp_false_task(2)
save_to_csv(tot_res, "1b_full_lookuptable_task0", header =["table_len","seed","is_correct","first_or_last_match","out","gold","func(n)","earliest_layer"])

  0%|          | 0/4 [00:00<?, ?it/s]

In [28]:
tot_res = exp_false_task(20, seed_start=2)
save_to_csv(tot_res, "1b_full_lookuptable_task1", header =["table_len","seed","is_correct","first_or_last_match","out","gold","func(n)","earliest_layer"])

  0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
tot_res = exp_false_task(20)
save_to_csv(tot_res, "1b_lookup_table_false_task_small0")

## Find the attn heads responsible

layer 18

In [13]:
with open("../results/full_lookuptable_task0.csv", "r") as f:
    df = pd.read_csv(f)

In [18]:
df

,Unnamed: 0,table_len,seed,is_correct,first_or_last_match,out,gold,func(n),earliest_layer
0,0,3,0,True,NaN,b,b,0,18
1,1,3,0,True,NaN,c,c,1,18
2,2,3,0,True,NaN,d,d,2,17
3,3,3,0,True,NaN,a,a,3,10
4,4,3,0,True,NaN,a,a,4,10
...,...,...,...,...,...,...,...,...,...
395,395,12,1,True,NaN,k,k,45,18
396,396,12,1,False,Match_First_Digit_False_Task,j,k,46,18
397,397,12,1,False,Match_First_Digit_False_Task,j,k,47,18
398,398,12,1,False,Match_First_Digit_False_Task,j,k,48,18


In [93]:
correct_data = df.loc[df['is_correct'] == True]
incorrect_data = df.loc[df['is_correct'] == False]
len(incorrect_data),len(correct_data)

(121, 279)

In [87]:
res = []
heads = []
for i in range(20):
    for h in range(48):
        heads.append((i, h))
for index, row in correct_data.iterrows():
    txt = noise_attn_heads(lm, 
                        make_lookup_table(row["table_len"], seed=int(row["seed"])) + "\n" + make_assert(int(row["func(n)"])), 
                        heads_to_noise=heads,
                        replace=True)
    if txt[0] == row["gold"]:
        res.append((True,txt[0], row["table_len"], row["seed"], row["is_correct"], row["first_or_last_match"], row["out"], row["gold"], row["func(n)"], row["earliest_layer"]))
    else:
        res.append((False, txt[0], row["table_len"], row["seed"], row["is_correct"], row["first_or_last_match"], row["out"], row["gold"], row["func(n)"], row["earliest_layer"]))
        
df_res = DataFrame.from_dict(res)
len(df_res.loc[df_res[0] == False])

In [99]:
res = []

for index, row in incorrect_data.iterrows():
    heads = []
    r = row["earliest_layer"]
    for i in range(r-2,r+3):
        for h in range(48):
            heads.append((i, h))
    txt = noise_attn_heads(lm, 
                        make_lookup_table(row["table_len"], seed=int(row["seed"])) + "\n" + make_assert(int(row["func(n)"])), 
                        heads_to_noise=heads,
                        replace=True)
    if txt[0] == row["gold"]:
        res.append((True,txt[0], row["table_len"], row["seed"], row["is_correct"], row["first_or_last_match"], row["out"], row["gold"], row["func(n)"], row["earliest_layer"]))
    else:
        res.append((False, txt[0], row["table_len"], row["seed"], row["is_correct"], row["first_or_last_match"], row["out"], row["gold"], row["func(n)"], row["earliest_layer"]))
        
df_res = DataFrame.from_dict(res)
len(df_res.loc[df_res[0] == True]), df_res

[(10, 0), (10, 1), (10, 2), (10, 3), (10, 4), (10, 5), (10, 6), (10, 7), (10, 8), (10, 9), (10, 10), (10, 11), (10, 12), (10, 13), (10, 14), (10, 15), (10, 16), (10, 17), (10, 18), (10, 19), (10, 20), (10, 21), (10, 22), (10, 23), (10, 24), (10, 25), (10, 26), (10, 27), (10, 28), (10, 29), (10, 30), (10, 31), (10, 32), (10, 33), (10, 34), (10, 35), (10, 36), (10, 37), (10, 38), (10, 39), (10, 40), (10, 41), (10, 42), (10, 43), (10, 44), (10, 45), (10, 46), (10, 47), (11, 0), (11, 1), (11, 2), (11, 3), (11, 4), (11, 5), (11, 6), (11, 7), (11, 8), (11, 9), (11, 10), (11, 11), (11, 12), (11, 13), (11, 14), (11, 15), (11, 16), (11, 17), (11, 18), (11, 19), (11, 20), (11, 21), (11, 22), (11, 23), (11, 24), (11, 25), (11, 26), (11, 27), (11, 28), (11, 29), (11, 30), (11, 31), (11, 32), (11, 33), (11, 34), (11, 35), (11, 36), (11, 37), (11, 38), (11, 39), (11, 40), (11, 41), (11, 42), (11, 43), (11, 44), (11, 45), (11, 46), (11, 47), (12, 0), (12, 1), (12, 2), (12, 3), (12, 4), (12, 5), (12, 

(0,
          0  1   2  3      4                             5  6  7   8   9
 0    False  d   3  0  False  Match_First_Digit_False_Task  d  a  27  12
 1    False  d   3  0  False  Match_First_Digit_False_Task  d  a  28  12
 2    False  b   3  1  False  Match_First_Digit_False_Task  b  d  27  18
 3    False  b   3  1  False  Match_First_Digit_False_Task  b  d  28  18
 4    False  g   6  0  False   Match_Last_Digit_False_Task  g  b  13  18
 ..     ... ..  .. ..    ...                           ... .. ..  ..  ..
 116  False  j  12  1  False  Match_First_Digit_False_Task  j  k  43  18
 117  False  j  12  1  False  Match_First_Digit_False_Task  j  k  46  18
 118  False  j  12  1  False  Match_First_Digit_False_Task  j  k  47  18
 119  False  j  12  1  False  Match_First_Digit_False_Task  j  k  48  18
 120  False  j  12  1  False  Match_First_Digit_False_Task  j  k  49  18
 
 [121 rows x 10 columns])

In [16]:
txt, ret = lm.trace_generate(make_lookup_table(12, seed=0) + "\n" + make_assert(48),
                  max_new_toks=1, 
                  do_greedy_decoding=True)

In [202]:
txt = noise_hidden_states(lm, 
                        make_lookup_table(3, seed=1) + "\n" + make_assert(27), 
                        tok_range_per_state=[(18,-5,-3)],
                        replace=True)
txt

torch.Size([1, 63, 6144])


['b']

In [189]:
lm.tokenizer.encode("18")

[35, 42]

In [205]:
print(make_lookup_table(12,0))

def func(i : int) -> str:
    if i == 6:
        return "e"
    elif i == 11:
        return "c"
    elif i == 0:
        return "b"
    elif i == 4:
        return "j"
    elif i == 7:
        return "m"
    elif i == 3:
        return "l"
    elif i == 2:
        return "h"
    elif i == 10:
        return "a"
    elif i == 5:
        return "f"
    elif i == 8:
        return "g"
    elif i == 9:
        return "k"
    elif i == 1:
        return "i"
    else:
        return "d"
